In [ ]:
# ============ IMPORTS ============
import pandas as pd
import numpy as np
import sys

# ============ LOAD CACHED DATA ============
processed_data_cache = pd.read_pickle('data_cache/processed_data.pkl')
bitcoin_price_and_features = processed_data_cache['bitcoin_price_and_features']

# Start with normalised features
labelling_df = processed_data_cache['features_norm']

# Add Close price (NOT normalised) for reference
labelling_df['Close'] = bitcoin_price_and_features[('Close', 'BTC-USD')]

labelling_df['Near_Peak'] = 0

labelling_df.to_csv('data_cache/labelling_df.csv', index=True)